In [1]:
from sklearn import svm
import numpy as np
import matplotlib.pyplot as plt
import cv2

/home/oliviermt/miniconda3/lib/python3.6/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [3]:
def define_classes(labels):
    classes = {}
    label = []
    c = 0
    for i,j in labels:
        if j.decode('utf-8') not in classes:
            classes[j.decode('utf-8')] = c
            c += 1
        label.append(classes[j.decode('utf-8')])    
    return classes, np.array(label)

In [4]:
def reshape_images(images):
    train_im = []  
    for i in range(images.shape[0]):
        train_im.append(np.array(images[i][1]))
    return np.array(train_im)

In [5]:
classes, train_labels = define_classes(np.genfromtxt('train_labels.csv',names=True, delimiter=',', dtype=[('Id', 'i8'), ('Category', 'S5')]))
train_images = reshape_images(np.load('Processed_andCleaned_train_images.npy', encoding='latin1'))
test_images = reshape_images(np.load('Processed_andCleaned_test_images.npy', encoding='latin1'))
train_images = train_images/np.max(train_images)
test_images = test_images/np.max(test_images)

In [6]:
def hu_moments(image):
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

In [17]:
train_features = np.zeros((len(train_images), 7))
test_features  = np.zeros((len(test_images),  7)) 
for i, image in enumerate(train_images):
    moments = hu_moments(image)
    train_features[i] = -np.sign(moments)*np.log(np.abs(moments))
for i, image in enumerate(test_images):
    moments = hu_moments(image)
    test_features[i] = -np.sign(moments)*np.log(np.abs(moments))

In [18]:
clf = svm.LinearSVC(penalty='l2', loss = 'squared_hinge', dual = False, tol = 0.0001, max_iter = 5000)

In [19]:
clf.fit(train_features, train_labels) 

LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=5000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [20]:
clf.score(train_features,train_labels)

0.2195

In [21]:
predictions = clf.predict(test_features)

In [26]:
import csv

In [27]:
classes = {3:'apple',21:'empty',23:'moustache',6:'mouth',30:'mug',19:'nail',
        13:'nose',22:'octagon',24:'paintbrush',25:'panda',26:'parrot',9:'peanut',16:'pear',
        20:'pencil',18:'penguin',17:'pillow',5:'pineapple',15:'pool',10:'rabbit',
        29:'rhinoceros',1:'rifle',8:'rollerskates',12:'sailboat',2:'scorpion',27:'screwdriver',
        0:'shovel',11:'sink',7:'skateboard',14:'skull',4:'spoon',28:'squiggle'}

In [31]:
with open('submission.csv', 'w') as submission:
    writer = csv.writer(submission, delimiter = ',',
            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for i, pred in enumerate(predictions):
        writer.writerow([str(i), classes[pred]])